# Подготовка данных для определения параметров модели, предсказывающей результаты госэкзамена по математике для каждого ученика в зависимости от условий жизни обучающихся

Импорт необходимых библиотек:

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.figure_factory as ff
from itertools import combinations
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.io as pio
pio.renderers.default='svg'

init_notebook_mode(connected=True)
cf.go_offline()

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

### Суть проекта — отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

    Проведем разведывательный анализ данных и определим параметры, влияющие на успеваемость, то есть на количество баллов по госэкзамену по математике 'score'

# I. Первичный отсмотр данных

In [2]:
df = pd.read_csv('stud_math.csv')
display(df.head(10))
df.info()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,course,mother,2.0,2.0,0.0,yes,no,no,no,yes,-6.0,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,other,mother,1.0,2.0,3.0,yes,no,NaN,no,yes,-6.0,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,home,mother,1.0,3.0,0.0,no,yes,yes,yes,yes,-9.0,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,home,father,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
5,GP,M,16,U,LE3,T,4.0,3.0,services,other,reputation,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,4.0,2.0,5.0,10.0,75.0
6,GP,M,16,NaN,LE3,T,2.0,2.0,other,other,home,mother,1.0,2.0,0.0,no,no,no,no,yes,-6.0,yes,yes,no,4.0,4.0,4.0,3.0,0.0,55.0
7,GP,F,17,U,GT3,A,4.0,4.0,other,teacher,home,mother,2.0,2.0,0.0,yes,yes,no,no,yes,-6.0,yes,no,no,4.0,1.0,4.0,1.0,6.0,30.0
8,GP,M,15,U,LE3,A,3.0,2.0,services,other,home,mother,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,yes,no,NaN,2.0,2.0,1.0,0.0,95.0
9,GP,M,15,U,NaN,NaN,3.0,4.0,other,other,home,mother,1.0,2.0,0.0,no,yes,yes,yes,yes,-6.0,yes,yes,no,5.0,5.0,1.0,5.0,0.0,75.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               395 non-null    object 
 1   sex                  395 non-null    object 
 2   age                  395 non-null    int64  
 3   address              378 non-null    object 
 4   famsize              368 non-null    object 
 5   Pstatus              350 non-null    object 
 6   Medu                 392 non-null    float64
 7   Fedu                 371 non-null    float64
 8   Mjob                 376 non-null    object 
 9   Fjob                 359 non-null    object 
 10  reason               378 non-null    object 
 11  guardian             364 non-null    object 
 12  traveltime           367 non-null    float64
 13  studytime            388 non-null    float64
 14  failures             373 non-null    float64
 15  schoolsup            386 non-null    obj

In [3]:
df.describe(include='all')

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
count,395,395,395.000000,378,368,350,392.000000,371.000000,376,359,378,364,367.000000,388.000000,373.000000,386,356,355,381,379,388.000000,375,361,364,368.000000,384.000000,387.000000,380.000000,383.000000,389.000000
unique,2,2,NaN,2,2,2,NaN,NaN,5,5,4,3,NaN,NaN,NaN,2,2,2,2,2,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN
top,GP,F,NaN,U,GT3,T,NaN,NaN,other,other,course,mother,NaN,NaN,NaN,no,yes,no,yes,yes,NaN,yes,yes,no,NaN,NaN,NaN,NaN,NaN,NaN
freq,349,208,NaN,295,261,314,NaN,NaN,133,197,137,250,NaN,NaN,NaN,335,219,200,195,300,NaN,356,305,240,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,16.696203,NaN,NaN,NaN,2.750000,2.614555,NaN,NaN,NaN,NaN,1.438692,2.038660,0.337802,NaN,NaN,NaN,NaN,NaN,-6.115979,NaN,NaN,NaN,3.937500,3.231771,3.105943,3.531579,7.279373,52.262211
std,NaN,NaN,1.276043,NaN,NaN,NaN,1.098127,2.228732,NaN,NaN,NaN,NaN,0.694181,0.842078,0.743135,NaN,NaN,NaN,NaN,NaN,2.526235,NaN,NaN,NaN,0.927277,0.993940,1.115896,1.396019,23.465197,22.919022
min,NaN,NaN,15.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,-12.000000,NaN,NaN,NaN,-1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,NaN,NaN,16.000000,NaN,NaN,NaN,2.000000,2.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,-6.000000,NaN,NaN,NaN,4.000000,3.000000,2.000000,3.000000,0.000000,40.000000
50%,NaN,NaN,17.000000,NaN,NaN,NaN,3.000000,2.000000,NaN,NaN,NaN,NaN,1.000000,2.000000,0.000000,NaN,NaN,NaN,NaN,NaN,-6.000000,NaN,NaN,NaN,4.000000,3.000000,3.000000,4.000000,4.000000,55.000000
75%,NaN,NaN,18.000000,NaN,NaN,NaN,4.000000,3.000000,NaN,NaN,NaN,NaN,2.000000,2.000000,0.000000,NaN,NaN,NaN,NaN,NaN,-3.000000,NaN,NaN,NaN,5.000000,4.000000,4.000000,5.000000,8.000000,70.000000


### Итак, мы имеем 30 колонок, из них 13 числовых столбцов и 17 строковых

Кроме того, мы видим, что в колонках "school", "sex", "age" отсутствуют пропуски

# II. Первичный анализ данных в столбцах

## school

In [4]:
df.school.describe()

count     395
unique      2
top        GP
freq      349
Name: school, dtype: object

In [5]:
df['school'].value_counts().iplot(kind='bar')

## sex

In [6]:
df.sex.describe()

count     395
unique      2
top         F
freq      208
Name: sex, dtype: object

In [7]:
df['sex'].value_counts().iplot(kind='bar')

## age

In [8]:
df.age.describe()

count    395.000000
mean      16.696203
std        1.276043
min       15.000000
25%       16.000000
50%       17.000000
75%       18.000000
max       22.000000
Name: age, dtype: float64

In [9]:
school_sex_age = px.parallel_categories(df, dimensions=['school', 'sex', 'age'],
                                        labels={'school': 'school', 'sex': 'sex', 'age': 'age'})
school_sex_age.show()

## address

In [10]:
df.address.describe()

count     378
unique      2
top         U
freq      295
Name: address, dtype: object

In [11]:
'''Заполним пропуски'''


def str_nan(col):
    return col.astype(str).apply(lambda x: None if x.strip() == '' else x)


df.address = str_nan(df.address)
df.address.describe()

count     395
unique      3
top         U
freq      295
Name: address, dtype: object

## famsize

In [12]:
df.famsize.describe()

count     368
unique      2
top       GT3
freq      261
Name: famsize, dtype: object

In [13]:
# также заменим пропуски на None
df.famsize = str_nan(df.famsize)
df.famsize.describe()

count     395
unique      3
top       GT3
freq      261
Name: famsize, dtype: object

In [14]:
address_school_sex_age_famsize = px.parallel_categories(df, dimensions=['address', 'school', 'sex', 'age', 'famsize'],
                                                        labels={'address': 'address', 'school': 'school', 'sex': 'sex', 'age': 'age', 'famsize': 'famsize'})
address_school_sex_age_famsize.show()

## Pstatus

In [15]:
df.Pstatus.describe()

count     350
unique      2
top         T
freq      314
Name: Pstatus, dtype: object

In [16]:
df.Pstatus = str_nan(df.Pstatus)
df.Pstatus.describe()

count     395
unique      3
top         T
freq      314
Name: Pstatus, dtype: object

Пропусков 11%, можем игнорировать - заменим на None. Однако, следует учитывать, что статус отношений родителей может умалчиваться более вероятно в случаях, когда родители живут раздельно, и респонденту не хочется говорить о личных сложностях

## Medu

In [17]:
df.Medu.describe()

count    392.000000
mean       2.750000
std        1.098127
min        0.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        4.000000
Name: Medu, dtype: float64

In [18]:
# заполним пропуски средним значением
df.Medu = df.Medu.fillna(round(df.Medu.median(), 1))
df.Medu.describe()

count    395.000000
mean       2.751899
std        1.094154
min        0.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        4.000000
Name: Medu, dtype: float64

In [19]:
df['Medu'].value_counts().iplot(kind='bar')

## Fedu

In [20]:
df.Fedu.describe()

count    371.000000
mean       2.614555
std        2.228732
min        0.000000
25%        2.000000
50%        2.000000
75%        3.000000
max       40.000000
Name: Fedu, dtype: float64

Очевидно, что значения более 4.0 не соответствуют здравому смыслу и являются ошибкой в данных. Хотя похоже, что просто при заполнении лишний 0 к цифре 4 приписали. Тем более, что это всего одно такое значение. Так что уберем этот лишний 0 и заполним пропуски медианным значением

In [21]:
df[df.Fedu > 4]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
11,GP,F,15,U,GT3,nan,2.0,40.0,services,other,reputation,father,3.0,3.0,0.0,no,yes,no,yes,yes,-9.0,yes,yes,no,5.0,2.0,2.0,4.0,4.0,60.0


In [22]:
df.Fedu = df.Fedu.apply(lambda x: 4 if x > 4 else x)
df.Fedu = df.Fedu.fillna(round(df.Fedu.median(), 1))
display(df.Fedu.describe())
df['Fedu'].value_counts().iplot(kind='bar')

count    395.000000
mean       2.486076
std        1.062362
min        0.000000
25%        2.000000
50%        2.000000
75%        3.000000
max        4.000000
Name: Fedu, dtype: float64

## Mjob

In [23]:
df.Mjob.describe()

count       376
unique        5
top       other
freq        133
Name: Mjob, dtype: object

In [24]:
# Заменим пропуски
df.Mjob = str_nan(df.Mjob)
df.Mjob.describe()

count       395
unique        6
top       other
freq        133
Name: Mjob, dtype: object

## Fjob

In [25]:
df.Fjob.describe()

count       359
unique        5
top       other
freq        197
Name: Fjob, dtype: object

In [26]:
# Заменим пропуски
df.Fjob = str_nan(df.Fjob)
df.Fjob.describe()

count       395
unique        6
top       other
freq        197
Name: Fjob, dtype: object

In [27]:
fig_mjob = px.histogram(df,
                        x="Mjob",
                        color="school")
fig_mjob.show()

## reason

In [28]:
df.reason.describe()

count        378
unique         4
top       course
freq         137
Name: reason, dtype: object

In [29]:
# заменим пропуски, выведем график
df.reason = str_nan(df.reason)
display(df.reason.describe())
df.reason.value_counts().iplot(kind='bar')

count        395
unique         5
top       course
freq         137
Name: reason, dtype: object

## guardian

In [30]:
df.guardian.describe()

count        364
unique         3
top       mother
freq         250
Name: guardian, dtype: object

In [31]:
# заменим пропуски, выведем график
df.guardian = str_nan(df.guardian)
display(df.guardian.describe())
df.guardian.value_counts().iplot(kind='bar')

count        395
unique         4
top       mother
freq         250
Name: guardian, dtype: object

## traveltime

In [32]:
df.traveltime.describe()

count    367.000000
mean       1.438692
std        0.694181
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: traveltime, dtype: float64

In [33]:
# заменим пропуски на медианное значение
df.traveltime = df.traveltime.fillna(round(df.traveltime.median(), 1))
display(df.traveltime.describe())
df.traveltime.value_counts().iplot(kind='bar')

count    395.000000
mean       1.407595
std        0.678490
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        4.000000
Name: traveltime, dtype: float64

## studytime

In [34]:
df.studytime.describe()

count    388.000000
mean       2.038660
std        0.842078
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        4.000000
Name: studytime, dtype: float64

In [35]:
# заменим пропуски медианным значением
# посмотрим на графике распределение studytime по школам
df.studytime = df.studytime.fillna(round(df.studytime.median(), 1))
display(df.studytime.describe())

px.histogram(df, x="studytime", color="school").show()

count    395.000000
mean       2.037975
std        0.834580
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        4.000000
Name: studytime, dtype: float64

## failures

In [36]:
df.failures.describe()

count    373.000000
mean       0.337802
std        0.743135
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        3.000000
Name: failures, dtype: float64

In [37]:
# Из условия под 0 у нас все значения кроме 1,2,3, заполним пропуски нулями
df.failures = df.failures.fillna(0)
display(df.failures.describe())
df.failures.value_counts().iplot(kind='bar')

count    395.000000
mean       0.318987
std        0.726244
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        3.000000
Name: failures, dtype: float64

## schoolsup

In [38]:
df.schoolsup.describe()

count     386
unique      2
top        no
freq      335
Name: schoolsup, dtype: object

In [39]:
# Заполним пропуски
df.schoolsup = str_nan(df.schoolsup)
df.schoolsup.describe()

count     395
unique      3
top        no
freq      335
Name: schoolsup, dtype: object

## famsup

In [40]:
df.famsup.describe()

count     356
unique      2
top       yes
freq      219
Name: famsup, dtype: object

In [41]:
# Заполним пропуски
df.famsup = str_nan(df.famsup)
df.famsup.describe()

count     395
unique      3
top       yes
freq      219
Name: famsup, dtype: object

## paid

In [42]:
df.paid.describe()

count     355
unique      2
top        no
freq      200
Name: paid, dtype: object

In [43]:
# Заполним пропуски
df.paid = str_nan(df.paid)
df.paid.describe()

count     395
unique      3
top        no
freq      200
Name: paid, dtype: object

## activities

In [44]:
df.activities.describe()

count     381
unique      2
top       yes
freq      195
Name: activities, dtype: object

In [45]:
# Заполним пропуски
df.activities = str_nan(df.activities)
df.activities.describe()

count     395
unique      3
top       yes
freq      195
Name: activities, dtype: object

## nursery

In [46]:
df.nursery.describe()

count     379
unique      2
top       yes
freq      300
Name: nursery, dtype: object

In [47]:
# Заполним пропуски
df.nursery = str_nan(df.nursery)
df.nursery.describe()

count     395
unique      3
top       yes
freq      300
Name: nursery, dtype: object

## studytime, granular

In [48]:
df['studytime, granular'].describe()

count    388.000000
mean      -6.115979
std        2.526235
min      -12.000000
25%       -6.000000
50%       -6.000000
75%       -3.000000
max       -3.000000
Name: studytime, granular, dtype: float64

In [49]:
df['studytime, granular'].value_counts().iplot(kind='bar')

In [50]:
df.drop(['studytime, granular'], inplace=True, axis=1)

## higher

Интересный параметр, очевидно, что студенты, которые не хотят получать высшее образование по каким либо причинам, вряд ли будут сильно стараться для госэкзамена

In [51]:
df.higher.describe()

count     375
unique      2
top       yes
freq      356
Name: higher, dtype: object

In [52]:
# Заполним пропуски
df.higher = str_nan(df.higher)
df.higher.describe()

count     395
unique      3
top       yes
freq      356
Name: higher, dtype: object

## internet

In [53]:
df.internet.describe()

count     361
unique      2
top       yes
freq      305
Name: internet, dtype: object

In [54]:
# Заполним пропуски
df.internet = str_nan(df.internet)
df.internet.describe()

count     395
unique      3
top       yes
freq      305
Name: internet, dtype: object

## romantic

In [55]:
df.romantic.describe()

count     364
unique      2
top        no
freq      240
Name: romantic, dtype: object

In [56]:
# Заполним пропуски
df.romantic = str_nan(df.romantic)
df.romantic.describe()

count     395
unique      3
top        no
freq      240
Name: romantic, dtype: object

## famrel

In [57]:
df.famrel.describe()

count    368.000000
mean       3.937500
std        0.927277
min       -1.000000
25%        4.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: famrel, dtype: float64

In [58]:
df[df.famrel < 1]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,health,absences,score
25,GP,F,16,U,GT3,nan,2.0,2.0,services,services,home,mother,1.0,1.0,2.0,no,yes,yes,no,no,yes,yes,no,-1.0,2.0,2.0,5.0,14.0,40.0


Очевидно, что значение -1 не соответсвует здравому смыслу и является ошибкой в заполнении данных, можем приедположить, что знак '-' перед единицей ввели случайно при заполнении. Так что уберем этот лишний минус и заполним пропуски медианным значением

In [59]:
df.famrel = df.famrel.apply(lambda x: abs(x) if x < 0 else x)
df.famrel = df.famrel.fillna(round(df.famrel.median(), 1))
display(df.famrel.describe(include='all'))
px.histogram(df, x="famrel", color="higher").show()

count    395.000000
mean       3.946835
std        0.872425
min        1.000000
25%        4.000000
50%        4.000000
75%        4.500000
max        5.000000
Name: famrel, dtype: float64

## freetime

In [60]:
df.freetime.describe()

count    384.000000
mean       3.231771
std        0.993940
min        1.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: freetime, dtype: float64

In [61]:
# заполеним пропуски медианным значением
df.freetime = df.freetime.fillna(round(df.freetime.median(), 1))
display(df.freetime.describe())
df.freetime.value_counts().iplot(kind='bar')

count    395.000000
mean       3.225316
std        0.980711
min        1.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: freetime, dtype: float64

## goout

In [62]:
df.goout.describe()

count    387.000000
mean       3.105943
std        1.115896
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: goout, dtype: float64

In [63]:
# заполним пропущенные значения медианным
df.goout = df.goout.fillna(round(df.goout.median(), 1))
display(df.goout.describe())
df.goout.value_counts().iplot(kind='bar')

count    395.000000
mean       3.103797
std        1.104610
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: goout, dtype: float64

## health

In [64]:
df.health.describe()

count    380.000000
mean       3.531579
std        1.396019
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: health, dtype: float64

In [65]:
# также заполним пропуски медианным значением
df.health = df.health.fillna(round(df.health.median(), 1))
display(df.health.describe())
df.health.value_counts().iplot(kind='bar')

count    395.000000
mean       3.549367
std        1.372118
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: health, dtype: float64

## absences

In [66]:
df.absences.describe()

count    383.000000
mean       7.279373
std       23.465197
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max      385.000000
Name: absences, dtype: float64

у нас есть выброс - студент с 385 пропусков и при этом с высоким баллом по экзамену, здоровье у этой девочки на 3, может это причина большого количества пропусков. Есть еще мальчик 212 пропусков, его баллы вроде средние. Это может сильно исказить нормальное распределение. Проверим на выбросы:

In [67]:
IQR = df.absences.quantile(0.75) - df.absences.quantile(0.25)
perc25 = df.absences.quantile(0.25)
perc75 = df.absences.quantile(0.75)
print('25-й перцентиль: {},'.format(perc25), '75-й перцентиль: {},'.format(perc75),
      "IQR: {}, ".format(IQR), "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))
display(df.absences.loc[df.absences.between(
    perc25 - 1.5*IQR, perc75 + 1.5*IQR)].describe())
px.histogram(df[df.absences.between(perc25 - 1.5*IQR, perc75 +
             1.5*IQR)], x="absences", color="sex").show()

25-й перцентиль: 0.0, 75-й перцентиль: 8.0, IQR: 8.0,  Границы выбросов: [-12.0, 20.0].


count    366.000000
mean       4.603825
std        4.965512
min        0.000000
25%        0.000000
50%        4.000000
75%        7.000000
max       20.000000
Name: absences, dtype: float64

In [68]:
px.histogram(df[~df.absences.between(perc25 - 1.5*IQR,
             perc75 + 1.5*IQR)], x="absences", color="sex").show()

Чтож, мы отобрали экстремально высокие данные. С одной стороны, экстремально высокие значения пропусков - это явный сигнал, что студент в группе риска, заставляет задуматься, почему он пропускает уроки и они важны для будущей модели, но могут сильно исказить среднюю картину. Вообще, у нас всего то 2 суперэкстремальных значения - 212 и 385. Если мы используем формулу с квантилями мы теряем слишком много значений от 20 до 100 пропусков. Так что пока просто отфильтруем значения менее 100 пропусков и заполним пропуски в данных медианным значением. 

In [69]:
df.absences = df.absences.fillna(round(df.absences.median(), 1))
df = df[df.absences < 100]
display(df.absences.describe())
df.absences.value_counts().iplot(kind='bar')

count    393.000000
mean       5.697201
std        7.969484
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max       75.000000
Name: absences, dtype: float64

## score

* score - важный параметр, так как именно его должна научиться предсказывать будущая модель

In [70]:
df.score.describe()

count    387.000000
mean      52.196382
std       22.914748
min        0.000000
25%       40.000000
50%       55.000000
75%       70.000000
max      100.000000
Name: score, dtype: float64

In [71]:
df[df.score.isnull()]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,health,absences,score
201,GP,F,16,U,GT3,T,2.0,3.0,other,other,reputation,mother,1.0,2.0,0.0,yes,yes,nan,yes,yes,yes,no,no,4.0,3.0,3.0,4.0,6.0,NaN
244,GP,F,18,U,GT3,T,2.0,1.0,other,other,course,other,2.0,3.0,0.0,no,yes,yes,no,no,yes,yes,yes,4.0,4.0,4.0,3.0,0.0,NaN
270,GP,F,19,nan,GT3,T,3.0,3.0,other,services,home,other,1.0,2.0,2.0,no,yes,yes,yes,yes,yes,yes,no,4.0,3.0,5.0,5.0,4.0,NaN
283,GP,F,18,U,GT3,T,1.0,1.0,other,other,home,mother,2.0,2.0,0.0,yes,nan,no,yes,yes,yes,yes,no,5.0,3.0,4.0,4.0,4.0,NaN
312,GP,M,19,U,GT3,T,1.0,2.0,other,services,course,other,1.0,2.0,1.0,no,no,no,no,no,yes,yes,no,4.0,5.0,3.0,4.0,3.0,NaN
350,MS,M,19,nan,GT3,T,1.0,1.0,other,services,home,other,3.0,2.0,3.0,no,no,no,no,nan,yes,yes,no,5.0,4.0,4.0,2.0,8.0,NaN


In [72]:
# заполним пропуски медианным значением
df.score = df.score.fillna(round(df.score.median(), 1))
df.describe(include='all')
df.score.iplot(kind='hist')

# Корреляционный анализ


In [73]:
corrs = df.corr()

figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)

figure.show()

#### Итак, мы видим положительную корреляцию со значением score: 
* absences - вообще, это странно, что больше пропусков=лучше балл, возможно, более умные студенты позволяют себе чаще пропускать уроки? тем более, что корреляция слабенькая
* freetime - чем больше свободного времени, тем лучше баллы на экзамене. Очевидно, что наличие свободного времени способствует нормальному психологическому состоянию и позволяет быть успешнее
* famrel - чем лучше отношения в семье, тем успешнее студент. Тут логично)
* studytime - чем больше дополнительного времени на учебу, тем лучше результат
* Fedu - образование отца. Логично, что у умных родителей и дети умнее
* Medu - образование матери. Интересно, что тут коэффициент корреляции гораздо выше чем у отца. Мама с хорошим образованием, видимо, отлично мотивирует своего ребенка
#### Также мы видим отрицательную корреляцию:
* health - интересно. Видимо, более физически здоровые студенты занимаются чем то кроме математики
* goout - чем больше времени студент проводит с друзьями, тем меньше остается на учебу
* failures - тут довольно сильная корреляция, количество внеучебных неудач довольно сильно снижает успеваемость
* traveltime - чем больше времени студент тратит на путь до школы, тем, видимо, у него меньше потом мотивации учиться
* age - можно предположить, что чем старше студент, тем более сложный у него учебный материал и естесственно, больше студентов начинают отставать


# Анализ номинативных переменных

### Номинативные переменные, которые осталось рассмотреть:  
'school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic'
* итого - 17 номинативных переменных

In [74]:
fig = make_subplots(rows=3, cols=3, subplot_titles=(
    'school', 'sex', 'address', 'famsize', 'Pstatus', 'reason', 'Mjob', 'Fjob', 'guardian'))
fig.add_trace(go.Box(x=df['school'], y=df['score']), row=1, col=1)
fig.add_trace(go.Box(x=df['sex'], y=df['score']), row=1, col=2)
fig.add_trace(go.Box(x=df['address'], y=df['score']), row=1, col=3)
fig.add_trace(go.Box(x=df['famsize'], y=df['score']), row=2, col=1)
fig.add_trace(go.Box(x=df['Pstatus'], y=df['score']), row=2, col=2)
fig.add_trace(go.Box(x=df['reason'], y=df['score']), row=2, col=3)
fig.add_trace(go.Box(x=df['Mjob'], y=df['score']), row=3, col=1)
fig.add_trace(go.Box(x=df['Fjob'], y=df['score']), row=3, col=2)
fig.add_trace(go.Box(x=df['guardian'], y=df['score']), row=3, col=3)

fig.show()

In [75]:
fig = make_subplots(rows=2, cols=4, subplot_titles=(
    'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic'))
fig.add_trace(go.Box(x=df['schoolsup'], y=df['score']), row=1, col=1)
fig.add_trace(go.Box(x=df['famsup'], y=df['score']), row=1, col=2)
fig.add_trace(go.Box(x=df['paid'], y=df['score']), row=1, col=3)
fig.add_trace(go.Box(x=df['activities'], y=df['score']), row=1, col=4)
fig.add_trace(go.Box(x=df['nursery'], y=df['score']), row=2, col=1)
fig.add_trace(go.Box(x=df['higher'], y=df['score']), row=2, col=2)
fig.add_trace(go.Box(x=df['internet'], y=df['score']), row=2, col=3)
fig.add_trace(go.Box(x=df['romantic'], y=df['score']), row=2, col=4)

fig.show()

Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения баллов  по различным параметрам неразличимы:

In [76]:
def get_stat_dif(column):
    cols = df.loc[:, column].value_counts()
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'],
                     df.loc[df.loc[:, column] == comb[1], 'score']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break
        print('Не найдены статистически значимые различия для колонки', column)

In [77]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    get_stat_dif(col)

Не найдены статистически значимые различия для колонки school
Не найдены статистически значимые различия для колонки sex
Не найдены статистически значимые различия для колонки address
Не найдены статистически значимые различия для колонки address
Не найдены статистически значимые различия для колонки address
Не найдены статистически значимые различия для колонки famsize
Не найдены статистически значимые различия для колонки famsize
Не найдены статистически значимые различия для колонки famsize
Не найдены статистически значимые различия для колонки Pstatus
Не найдены статистически значимые различия для колонки Pstatus
Не найдены статистически значимые различия для колонки Pstatus
Не найдены статистически значимые различия для колонки Mjob
Не найдены статистически значимые различия для колонки Mjob
Не найдены статистически значимые различия для колонки Mjob
Не найдены статистически значимые различия для колонки Mjob
Не найдены статистически значимые различия для колонки Mjob
Не найдены с

### Итак, ни одна из номинативных переменных не представляет интереса для построения модели.
##### В нашем случае важные переменные, которые, возможно, оказывают влияние на баллы за экзамен, это:    'age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'health', 'absences'
Оставим их для дальнейшего построения модели

In [78]:
df_for_model = df.loc[:, ['age', 'Medu', 'Fedu', 'traveltime', 'studytime',
                          'failures', 'famrel', 'freetime', 'goout', 'health', 'absences']]
df_for_model.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,health,absences
0,18,4.0,4.0,2.0,2.0,0.0,4.0,3.0,4.0,3.0,6.0
1,17,1.0,1.0,1.0,2.0,0.0,5.0,3.0,3.0,3.0,4.0
2,15,1.0,1.0,1.0,2.0,3.0,4.0,3.0,2.0,3.0,10.0
3,15,4.0,2.0,1.0,3.0,0.0,3.0,2.0,2.0,5.0,2.0
4,16,3.0,3.0,1.0,2.0,0.0,4.0,3.0,2.0,5.0,4.0


# Выводы

**Итак, в результате EDA для анализа влияния условий жизни школьников на успеваемость по математике были получены следующие выводы:**

    1. Нет пустых значений только в колонках 'school', 'sex', 'age'
    2. В остальных колонках пропусков до 10%, в строковых типах мы заполнили пропуски значением None, в количественных переменных заполнили пропуски медианным значением, в колонке failures заполнили пропуски нулями в соответсвии с условием
    3. Выбросы найдены в столбцах Fedu, famrel, absences, данные достаточно чистые
    4. Самые важные параметры, которые предлагается использовать в дальнейшем для построения модели:          
   **параметры с положительной корреляцией**
   
    * absences -  возможно, более умные студенты позволяют себе чаще пропускать уроки? 
    * freetime - чем больше свободного времени, тем лучше баллы на экзамене. Очевидно, что наличие свободного времени способствует нормальному психологическому состоянию и позволяет быть успешнее
    * famrel - чем лучше отношения в семье, тем успешнее студент. Тут логично)
    * studytime - чем больше дополнительного времени на учебу, тем лучше результат
    * Fedu - образование отца. Логично, что у умных родителей и дети умнее
    * Medu - образование матери. Интересно, что тут коэффициент корреляции гораздо выше чем у отца. Мама с хорошим образованием, видимо, отлично мотивирует своего ребенка    
   **параметры с отрицательной корреляцией:**
    
    * health - похоже, что более физически здоровые студенты занимаются чем то кроме математики
    * goout - чем больше времени студент проводит с друзьями, тем меньше остается на учебу
    * failures - тут довольно сильная корреляция, количество внеучебных неудач довольно сильно снижает успеваемость
    * traveltime - чем больше времени студент тратит на путь до школы, тем, видимо, у него меньше потом мотивации учиться
    * age - можно предположить, что чем старше студент, тем более сложный у него учебный материал и естесственно, больше студентов начинают отставать